In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Importaciones

**PyTorch (torch, nn, DataLoader):** Para crear y entrenar redes neuronales.

**numpy, pandas:** Para manejar y analizar datos numéricos y tablas.
re, string, math: Para trabajar con texto y operaciones matemáticas.

**collections:** Para contar y organizar datos fácilmente.

**warnings:** Para ocultar advertencias.

**nltk:** Para procesar y dividir texto en palabras.

**BLEU y ROUGE:** Para evaluar la calidad de textos generados por el modelo.

**subprocess:** Para instalar paquetes automáticamente si faltan.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import re
import string
import math
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings('ignore')

# Importaciones para métricas
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
try:
    from rouge_score import rouge_scorer
except ImportError:
    import subprocess
    subprocess.check_call(['pip', 'install', 'rouge-score'])
    from rouge_score import rouge_scorer

# Descargas necesarias de NLTK
for package in ['punkt', 'punkt_tab', 'wordnet']:
    try:
        nltk.download(package, quiet=True)
    except:
        pass

# Configuración de evaluación
smooth_function = SmoothingFunction().method2
evaluador_rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)

# Definición de semilla reproductibilidad

En este bloque se define y utiliza una función para establecer una semilla aleatoria fija en Python, NumPy y PyTorch. Esto asegura que los resultados de los experimentos sean reproducibles, es decir, que al ejecutar el código varias veces se obtengan los mismos resultados. También se configuran opciones adicionales para garantizar el determinismo en los cálculos y se fija la variable de entorno correspondiente.


In [ ]:
# Definir semilla antes de usarla
SEMILLA_ALEATORIA = 42

def establecer_semilla_reproducibilidad(semilla=42):
    """
    Establece semillas aleatorias para reproducibilidad completa
    """
    # Python
    import random
    random.seed(semilla)

    # NumPy
    np.random.seed(semilla)

    # PyTorch
    torch.manual_seed(semilla)
    torch.cuda.manual_seed(semilla)
    torch.cuda.manual_seed_all(semilla)

    # Configuración adicional para determinismo
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Variables de entorno
    import os
    os.environ['PYTHONHASHSEED'] = str(semilla)

    print(f"🌱 Semillas establecidas para reproducibilidad (semilla={semilla})")

# Establecer reproducibilidad
establecer_semilla_reproducibilidad(SEMILLA_ALEATORIA)


🌱 Semillas establecidas para reproducibilidad (semilla=42)


# Definición de cada parametro de entrenamiento

**DISPOSITIVO:**
Indica si se usará la GPU ("cuda") o la CPU para entrenar y ejecutar el modelo, dependiendo de la disponibilidad.

**TAMAÑO_LOTE (batch size):**
Número de ejemplos que se procesan juntos en cada paso de entrenamiento. Un valor mayor puede acelerar el entrenamiento, pero requiere más memoria.

**DIM_EMBEDDING:**
Dimensión de los vectores de embedding, es decir, el tamaño de la representación numérica de cada palabra o token.

**CABEZAS_ATENCION:**
Número de "cabezas" en la capa de atención múltiple del Transformer. Más cabezas permiten al modelo enfocarse en diferentes partes de la secuencia simultáneamente.

**CAPAS_TRANSFORMER:**
Cantidad de capas (bloques) del modelo Transformer. Más capas pueden aumentar la capacidad del modelo para aprender patrones complejos.

**LONGITUD_MAXIMA:**
Longitud máxima permitida para las secuencias de entrada o salida. Las secuencias más largas se recortan o rellenan hasta este tamaño.

**DROPOUT:**
Proporción de neuronas que se "apagan" aleatoriamente durante el entrenamiento para evitar el sobreajuste.

**TASA_APRENDIZAJE (learning rate):**
Velocidad con la que el modelo ajusta sus parámetros durante el entrenamiento. Un valor adecuado es clave para un buen aprendizaje.

**NUM_EPOCAS:**
Número de veces que el modelo recorre todo el conjunto de datos de entrenamiento.

**FACTOR_GRAD_CLIP:**
Límite máximo para el valor de los gradientes durante el entrenamiento, evitando que sean demasiado grandes y causen inestabilidad.

**UMBRAL_FRECUENCIA:**
Frecuencia mínima con la que una palabra debe aparecer en el corpus para ser incluida en el vocabulario del modelo.

In [ ]:
# === Parámetros del Sistema de Generación ===

# Configuración del hardware
DISPOSITIVO = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hiperparámetros del modelo
TAMAÑO_LOTE = 64
DIM_EMBEDDING = 512
CABEZAS_ATENCION = 4
CAPAS_TRANSFORMER = 3
LONGITUD_MAXIMA = 128
DROPOUT = 0.1

# Parámetros de entrenamiento
TASA_APRENDIZAJE = 2e-4
NUM_EPOCAS = 45
FACTOR_GRAD_CLIP = 1.0
UMBRAL_FRECUENCIA = 3

print(f"🚀 Dispositivo de procesamiento: {DISPOSITIVO}")
print(f"📊 Configuración: {TAMAÑO_LOTE} batch, {DIM_EMBEDDING} dim, {CABEZAS_ATENCION} heads")

🚀 Dispositivo de procesamiento: cuda
📊 Configuración: 64 batch, 512 dim, 4 heads


#Clase Vocabulario

Esta clase se encarga de construir y gestionar el vocabulario del modelo. Permite convertir palabras o símbolos en índices numéricos y viceversa, lo cual es esencial para que el modelo pueda trabajar con texto.

__init__: Inicializa el diccionario y el tokenizador.

__len__: Devuelve cuántos tokens hay en el diccionario.

**construir_diccionario:** Crea el vocabulario a partir de textos, solo con palabras frecuentes.

**procesar_texto:** Limpia y divide el texto en tokens.

**convertir_a_indices:** Convierte un texto en una lista de números (índices de tokens).

In [ ]:
class DiccionarioTokens:
    def __init__(self, min_frecuencia=3):
        # Tokens especiales con diferentes símbolos
        self.indice_a_token = {
            0: "[PAD]",
            1: "[INICIO]",
            2: "[FIN]",
            3: "[DESCONOCIDO]",
            4: ">>>"
        }
        self.token_a_indice = {v: k for k, v in self.indice_a_token.items()}
        self.min_frecuencia = min_frecuencia
        self.tokenizador = RegexpTokenizer(r'\w+|[^\w\s]+')

    def __len__(self):
        return len(self.indice_a_token)

    def construir_diccionario(self, lista_textos):
        contador_palabras = Counter()
        indice_actual = len(self.indice_a_token)

        # Primera pasada: contar frecuencias
        for texto in lista_textos:
            tokens = self.procesar_texto(texto)
            contador_palabras.update(tokens)

        # Segunda pasada: agregar palabras frecuentes
        for palabra, frecuencia in contador_palabras.items():
            if frecuencia >= self.min_frecuencia and palabra not in self.token_a_indice:
                self.token_a_indice[palabra] = indice_actual
                self.indice_a_token[indice_actual] = palabra
                indice_actual += 1

    def procesar_texto(self, texto):
        # Tokenización personalizada con regex
        texto = texto.lower().strip()
        tokens = self.tokenizador.tokenize(texto)
        return [t for t in tokens if t and not t.isspace()]

    def convertir_a_indices(self, texto):
        tokens = self.procesar_texto(texto)
        desconocido_idx = self.token_a_indice["[DESCONOCIDO]"]
        return [self.token_a_indice.get(token, desconocido_idx) for token in tokens]

# clase ConjuntoDatosReseñas:
Esta clase prepara y organiza los datos de reseñas para entrenar el modelo. Lee un archivo CSV con reseñas y categorías, limpia los datos, construye el vocabulario necesario y convierte cada ejemplo en una secuencia de números (índices de tokens). Además, agrega tokens especiales, aplica padding para igualar la longitud de las secuencias y genera los pares de entrada y salida que necesita el modelo para aprender a predecir el siguiente token en una secuencia.



*   __init__:
Carga los datos desde un archivo CSV, elimina filas vacías, crea el diccionario de tokens y prepara los textos combinando la categoría y la reseña.

* __len__:
Devuelve cuántos ejemplos hay en el conjunto de datos.


*  __getitem__:
Toma una fila, la convierte en texto, la tokeniza y la transforma en una secuencia de índices con tokens especiales. Aplica padding y prepara los tensores de entrada y salida para el modelo.






In [ ]:
class ConjuntoDatosReseñas(Dataset):
    def __init__(self, archivo_csv, longitud_max=LONGITUD_MAXIMA):
        # Cargar y preparar datos
        self.datos = pd.read_csv(archivo_csv)
        self.datos = self.datos.dropna(subset=['Review Text', 'Class Name'])
        self.longitud_max = longitud_max

        # Crear diccionario de tokens
        self.diccionario = DiccionarioTokens(min_frecuencia=UMBRAL_FRECUENCIA)

        # Preparar textos combinados con separador diferente
        textos_completos = []
        for _, fila in self.datos.iterrows():
            categoria = str(fila['Class Name']).lower().strip()
            reseña = str(fila['Review Text']).lower().strip()
            texto_combinado = f"{categoria} >>> {reseña}"
            textos_completos.append(texto_combinado)

        self.diccionario.construir_diccionario(textos_completos)
        print(f"📖 Tamaño del vocabulario: {len(self.diccionario)}")

    def __len__(self):
        return len(self.datos)

    def __getitem__(self, indice):
        fila = self.datos.iloc[indice]
        categoria = str(fila['Class Name']).lower().strip()
        reseña = str(fila['Review Text']).strip()

        # Combinar con el nuevo separador
        texto_completo = f"{categoria} >>> {reseña}"

        # Convertir a índices con tokens especiales
        indices = [self.diccionario.token_a_indice["[INICIO]"]]
        indices.extend(self.diccionario.convertir_a_indices(texto_completo))
        indices.append(self.diccionario.token_a_indice["[FIN]"])

        # Aplicar padding
        tensor_padding = torch.zeros(self.longitud_max, dtype=torch.long)
        longitud_secuencia = min(len(indices), self.longitud_max)
        tensor_padding[:longitud_secuencia] = torch.tensor(indices[:longitud_secuencia])

        # Preparar entrada y salida (shift by 1)
        entrada = tensor_padding[:-1]
        salida = tensor_padding[1:]

        return entrada, salida

# clase CodificacionPosicional:
Esta clase implementa la codificación posicional, una técnica esencial en los modelos Transformer para que el modelo sepa el orden de las palabras en una secuencia. Genera una matriz con valores basados en funciones seno y coseno, que se suma a los embeddings de las palabras. Así, cada posición en la secuencia tiene una representación única. Además, aplica dropout para ayudar a evitar el sobreajuste.


*   __init__:
Inicializa la clase, crea la matriz de codificación posicional usando funciones seno y coseno para cada posición y dimensión, y la guarda como un buffer (no se entrena).

* **forward:**
Suma la codificación posicional a los embeddings de entrada y aplica dropout. Así, cada token tiene información sobre su posición en la secuencia.



In [ ]:
class CodificacionPosicional(nn.Module):
    def __init__(self, dimension_embed, longitud_max=LONGITUD_MAXIMA, base=10000):
        super().__init__()
        self.dimension_embed = dimension_embed
        self.dropout = nn.Dropout(DROPOUT)

        # Crear matriz de codificación posicional
        matriz_pe = torch.zeros(longitud_max, dimension_embed)
        posiciones = torch.arange(0, longitud_max).unsqueeze(1).float()

        # Calcular divisores para frecuencias
        indices_pares = torch.arange(0, dimension_embed, 2).float()
        divisor = torch.pow(base, indices_pares / dimension_embed)

        # Aplicar funciones seno y coseno
        matriz_pe[:, 0::2] = torch.sin(posiciones / divisor)
        matriz_pe[:, 1::2] = torch.cos(posiciones / divisor)

        # Registrar como buffer (no se entrena)
        self.register_buffer('codificacion_pos', matriz_pe.unsqueeze(0))

    def forward(self, embeddings):
        # Agregar codificación posicional y aplicar dropout
        longitud_seq = embeddings.size(1)
        salida = embeddings + self.codificacion_pos[:, :longitud_seq, :]
        return self.dropout(salida)

# clase ModeloGeneradorTexto:
Esta clase define el modelo generador de texto basado en la arquitectura Transformer. Incluye una capa de embeddings para convertir los tokens en vectores, una codificación posicional para indicar el orden de los tokens, y un decodificador Transformer compuesto por varias capas de atención y feedforward. El modelo utiliza máscaras causales para asegurar que cada posición solo pueda "ver" los tokens anteriores, lo que es esencial para la generación de texto. Finalmente, normaliza la salida y la proyecta al tamaño del vocabulario para predecir el siguiente token.

In [ ]:
class ModeloGeneradorTexto(nn.Module):
    def __init__(self, tamaño_vocabulario, dim_modelo, num_cabezas, num_capas, dropout=DROPOUT):
        super().__init__()

        # Capas de embedding
        self.capa_embedding = nn.Embedding(tamaño_vocabulario, dim_modelo)
        self.escala_embedding = math.sqrt(dim_modelo)
        self.codificador_posicion = CodificacionPosicional(dim_modelo)

        # Configuración del decodificador transformer
        self.dropout_entrada = nn.Dropout(dropout)
        configuracion_capa = nn.TransformerDecoderLayer(
            d_model=dim_modelo,
            nhead=num_cabezas,
            dim_feedforward=dim_modelo * 4,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.decodificador = nn.TransformerDecoder(configuracion_capa, num_layers=num_capas)

        # Capa de salida
        self.normalizacion_final = nn.LayerNorm(dim_modelo)
        self.proyeccion_salida = nn.Linear(dim_modelo, tamaño_vocabulario)

        # Inicialización de pesos
        self._inicializar_pesos()

    def _inicializar_pesos(self):
        # Inicialización Xavier para mejor convergencia
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def crear_mascara_causal(self, tamaño):
        # Crear máscara triangular superior para atención causal
        mascara = torch.triu(torch.ones(tamaño, tamaño), diagonal=1)
        return mascara.bool().to(self.capa_embedding.weight.device)

    def forward(self, tokens_entrada, mascara_padding=None):
        tamaño_secuencia = tokens_entrada.size(1)

        # Embeddings con escalamiento
        embeddings = self.capa_embedding(tokens_entrada) * self.escala_embedding
        embeddings_con_pos = self.codificador_posicion(embeddings)
        embeddings_con_pos = self.dropout_entrada(embeddings_con_pos)

        # Crear máscara causal
        mascara_atencion = self.crear_mascara_causal(tamaño_secuencia)

        # Pasar por el decodificador
        salida_decodificador = self.decodificador(
            tgt=embeddings_con_pos,
            memory=embeddings_con_pos,
            tgt_mask=mascara_atencion,
            memory_mask=mascara_atencion,
            tgt_key_padding_mask=mascara_padding,
            memory_key_padding_mask=mascara_padding
        )

        # Normalización y proyección final
        salida_normalizada = self.normalizacion_final(salida_decodificador)
        logits = self.proyeccion_salida(salida_normalizada)

        return logits



* __init__:
Inicializa todas las capas del modelo: embeddings, codificación posicional, decodificador Transformer, normalización y capa de salida. También prepara la inicialización de los pesos.

* **_inicializar_pesos:**
Aplica la inicialización Xavier a los pesos del modelo para mejorar la convergencia durante el entrenamiento.

* **crear_mascara_causal:**
Genera una máscara triangular que impide que el modelo vea tokens futuros durante la generación de texto (atención causal).

* **forward:**
Define el paso hacia adelante del modelo: convierte los tokens en embeddings, suma la codificación posicional, aplica dropout, crea la máscara causal, pasa los datos por el decodificador y finalmente normaliza y proyecta la salida para obtener las predicciones.



# Función entrenar_epoca:
Esta función realiza una época completa de entrenamiento del modelo. Recorre los lotes de datos, mueve los tensores al dispositivo adecuado (CPU o GPU), crea la máscara de padding, realiza la predicción del modelo y calcula la pérdida. Además, implementa la acumulación de gradientes para optimizar el uso de memoria, aplica "gradient clipping" para evitar inestabilidades y actualiza los pesos del modelo con el optimizador. Al final, devuelve la pérdida promedio de la época.

In [ ]:
def entrenar_epoca(modelo, cargador_datos, optimizador, funcion_perdida, indice_pad, acumulacion_grad=1):
    modelo.train()
    perdida_acumulada = 0.0
    num_actualizaciones = 0

    for batch_idx, (entrada, objetivo) in enumerate(cargador_datos):
        # Mover datos a dispositivo
        entrada = entrada.to(DISPOSITIVO)
        objetivo = objetivo.to(DISPOSITIVO)

        # Crear máscara de padding
        mascara_padding = (entrada == indice_pad)

        # Forward pass
        predicciones = modelo(entrada, mascara_padding)

        # Calcular pérdida
        perdida = funcion_perdida(
            predicciones.reshape(-1, predicciones.size(-1)),
            objetivo.reshape(-1)
        )

        # Normalizar pérdida para acumulación de gradientes
        perdida = perdida / acumulacion_grad
        perdida.backward()

        # Actualizar pesos cada N pasos
        if (batch_idx + 1) % acumulacion_grad == 0:
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(modelo.parameters(), FACTOR_GRAD_CLIP)

            # Paso del optimizador
            optimizador.step()
            optimizador.zero_grad()
            num_actualizaciones += 1

        perdida_acumulada += perdida.item() * acumulacion_grad

    return perdida_acumulada / len(cargador_datos)

# Función generar_texto_reseña:
Esta función genera automáticamente una reseña de producto usando el modelo entrenado. Comienza con el tipo de producto y un separador especial, y va prediciendo palabra por palabra hasta alcanzar la longitud máxima o encontrar el token de fin. Utiliza técnicas de muestreo probabilístico (top-k y temperatura) para hacer la generación más variada y natural. Finalmente, decodifica los índices generados a texto, elimina los tokens especiales y devuelve solo la reseña generada.

In [ ]:
def generar_texto_reseña(modelo, tipo_producto, diccionario, longitud_max=50, temperatura=1.0, top_k=50):
    modelo.eval()

    # Preparar tokens iniciales
    texto_inicial = f"{tipo_producto.lower()} >>>"
    indices_tokens = [diccionario.token_a_indice["[INICIO]"]]
    indices_tokens.extend(diccionario.convertir_a_indices(texto_inicial))

    with torch.no_grad():
        for _ in range(longitud_max):
            # Preparar entrada
            tensor_entrada = torch.tensor(indices_tokens, dtype=torch.long).unsqueeze(0).to(DISPOSITIVO)

            # Obtener predicciones
            salida = modelo(tensor_entrada)
            logits_siguiente = salida[0, -1, :] / temperatura

            # Aplicar top-k sampling
            if top_k > 0:
                valores, indices = torch.topk(logits_siguiente, top_k)
                logits_siguiente[logits_siguiente < valores[-1]] = -float('Inf')

            # Muestreo probabilístico
            probabilidades = F.softmax(logits_siguiente, dim=-1)
            token_predicho = torch.multinomial(probabilidades, 1).item()

            indices_tokens.append(token_predicho)

            # Verificar token de fin
            if token_predicho == diccionario.token_a_indice["[FIN]"]:
                break

    # Decodificar tokens a texto
    tokens_finales = indices_tokens[1:]  # Omitir [INICIO]
    if diccionario.token_a_indice["[FIN]"] in tokens_finales:
        idx_fin = tokens_finales.index(diccionario.token_a_indice["[FIN]"])
        tokens_finales = tokens_finales[:idx_fin]

    # Convertir a palabras
    palabras = []
    for idx in tokens_finales:
        if idx in diccionario.indice_a_token:
            token = diccionario.indice_a_token[idx]
            if token not in ["[INICIO]", "[FIN]", "[PAD]"]:
                palabras.append(token)

    texto_generado = ' '.join(palabras)

    # Limpiar y retornar solo la reseña
    if ">>>" in texto_generado:
        partes = texto_generado.split(">>>", 1)
        return partes[1].strip() if len(partes) > 1 else texto_generado
    return texto_generado

# Función evaluar_calidad_generacion:
Esta función evalúa la calidad de los textos generados por el modelo utilizando métricas automáticas. Para varias muestras del conjunto de datos, genera un texto, lo compara con el texto real y calcula las métricas BLEU y ROUGE (ROUGE-1, ROUGE-2 y ROUGE-L), que miden la similitud entre el texto generado y el de referencia. Muestra los resultados de cada muestra y, al final, presenta un resumen con los promedios y desviaciones estándar de las métricas obtenidas.

In [ ]:
def evaluar_calidad_generacion(modelo, conjunto_datos, diccionario, num_muestras=15, temperatura=0.8):
    # Listas para almacenar métricas
    metricas_bleu = []
    metricas_rouge1 = []
    metricas_rouge2 = []
    metricas_rougeL = []

    print("🔍 Evaluando calidad del modelo generativo...\n")

    for idx in range(min(num_muestras, len(conjunto_datos))):
        # Obtener datos de referencia
        tipo_articulo = conjunto_datos.datos.iloc[idx]['Class Name']
        texto_referencia = conjunto_datos.datos.iloc[idx]['Review Text']

        # Generar texto
        texto_generado = generar_texto_reseña(
            modelo, tipo_articulo, diccionario,
            longitud_max=60, temperatura=temperatura
        )

        # Tokenizar para BLEU
        tokens_referencia = word_tokenize(texto_referencia.lower())
        tokens_generados = word_tokenize(texto_generado.lower())

        # Calcular BLEU
        puntuacion_bleu = sentence_bleu(
            [tokens_referencia],
            tokens_generados,
            smoothing_function=smooth_function
        )

        # Calcular ROUGE
        puntuaciones_rouge = evaluador_rouge.score(texto_referencia, texto_generado)

        # Almacenar métricas
        metricas_bleu.append(puntuacion_bleu)
        metricas_rouge1.append(puntuaciones_rouge['rouge1'].fmeasure)
        metricas_rouge2.append(puntuaciones_rouge['rouge2'].fmeasure)
        metricas_rougeL.append(puntuaciones_rouge['rougeL'].fmeasure)

        # Mostrar resultados
        print(f"📝 Muestra {idx+1}/{num_muestras}")
        print(f"   Categoría: {tipo_articulo}")
        print(f"   BLEU: {puntuacion_bleu:.3f} | R-1: {puntuaciones_rouge['rouge1'].fmeasure:.3f} | R-2: {puntuaciones_rouge['rouge2'].fmeasure:.3f} | R-L: {puntuaciones_rouge['rougeL'].fmeasure:.3f}")
        print(f"   Referencia: {texto_referencia[:100]}...")
        print(f"   Generado: {texto_generado[:100]}...")
        print("-" * 80)

    # Estadísticas finales
    print("\n📊 RESUMEN DE MÉTRICAS")
    print("=" * 50)
    print(f"BLEU promedio:    {np.mean(metricas_bleu):.4f} (±{np.std(metricas_bleu):.4f})")
    print(f"ROUGE-1 promedio: {np.mean(metricas_rouge1):.4f} (±{np.std(metricas_rouge1):.4f})")
    print(f"ROUGE-2 promedio: {np.mean(metricas_rouge2):.4f} (±{np.std(metricas_rouge2):.4f})")
    print(f"ROUGE-L promedio: {np.mean(metricas_rougeL):.4f} (±{np.std(metricas_rougeL):.4f})")

    return {
        'bleu': metricas_bleu,
        'rouge1': metricas_rouge1,
        'rouge2': metricas_rouge2,
        'rougeL': metricas_rougeL
    }

# Entrenamiento del Modelo

En este bloque se realiza toda la preparación y ejecución del proceso de entrenamiento del modelo generador de texto. Se cargan los datos desde un archivo CSV, se construye el conjunto de datos y el dataloader, y se configura el vocabulario. Luego, se inicializa el modelo Transformer, el optimizador, el scheduler para ajustar la tasa de aprendizaje y la función de pérdida con suavizado de etiquetas.

A continuación, se ejecuta el bucle de entrenamiento durante varias épocas, mostrando el progreso, la pérdida y el tiempo de cada época. El modelo se guarda automáticamente cuando mejora la pérdida, y cada cierto número de épocas se generan ejemplos de texto para monitorear la calidad del modelo.

In [ ]:
# === CONFIGURACIÓN DEL SISTEMA DE ENTRENAMIENTO ===

# Rutas de archivos
RUTA_DATOS = "/content/drive/MyDrive/ET_deep_learning/Reviews.csv"
RUTA_GUARDAR_MODELO = "/content/drive/MyDrive/ET_deep_learning/modelo_generativo_v2.pth"

# Verificar disponibilidad de datos
print("📁 Cargando conjunto de datos...")
datos_brutos = pd.read_csv(RUTA_DATOS)
print(f"✓ Total de registros encontrados: {len(datos_brutos)}")

# Crear dataset y dataloader
conjunto_entrenamiento = ConjuntoDatosReseñas(RUTA_DATOS)
cargador_entrenamiento = DataLoader(
    conjunto_entrenamiento,
    batch_size=TAMAÑO_LOTE,
    shuffle=True,
    num_workers=2 if DISPOSITIVO.type == 'cuda' else 0,
    pin_memory=True if DISPOSITIVO.type == 'cuda' else False
)

# Configuración del vocabulario
diccionario_vocabulario = conjunto_entrenamiento.diccionario
indice_padding = diccionario_vocabulario.token_a_indice["[PAD]"]
tamaño_vocabulario = len(diccionario_vocabulario)

print(f"📚 Tamaño del vocabulario construido: {tamaño_vocabulario}")
print(f"🔢 Total de lotes por época: {len(cargador_entrenamiento)}")

# === INICIALIZACIÓN DEL MODELO ===
print("\n🏗️ Construyendo arquitectura del modelo...")
modelo_generativo = ModeloGeneradorTexto(
    tamaño_vocabulario=tamaño_vocabulario,
    dim_modelo=DIM_EMBEDDING,
    num_cabezas=CABEZAS_ATENCION,
    num_capas=CAPAS_TRANSFORMER,
    dropout=DROPOUT
).to(DISPOSITIVO)

# Contar parámetros
total_parametros = sum(p.numel() for p in modelo_generativo.parameters())
parametros_entrenables = sum(p.numel() for p in modelo_generativo.parameters() if p.requires_grad)
print(f"📊 Total de parámetros: {total_parametros:,}")
print(f"📊 Parámetros entrenables: {parametros_entrenables:,}")

# Configuración del optimizador con scheduler
optimizador = torch.optim.AdamW(
    modelo_generativo.parameters(),
    lr=TASA_APRENDIZAJE,
    betas=(0.9, 0.98),
    eps=1e-9,
    weight_decay=0.01
)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizador,
    T_max=NUM_EPOCAS,
    eta_min=TASA_APRENDIZAJE * 0.1
)

# Función de pérdida con label smoothing
criterio_perdida = nn.CrossEntropyLoss(
    ignore_index=indice_padding,
    label_smoothing=0.1
)

# === BUCLE DE ENTRENAMIENTO ===
print("\n🚀 INICIANDO PROCESO DE ENTRENAMIENTO")
print("=" * 60)

historial_perdidas = []
mejor_perdida = float('inf')

for epoca in range(1, NUM_EPOCAS + 1):
    tiempo_inicio = torch.cuda.Event(enable_timing=True) if DISPOSITIVO.type == 'cuda' else None
    tiempo_fin = torch.cuda.Event(enable_timing=True) if DISPOSITIVO.type == 'cuda' else None

    if tiempo_inicio:
        tiempo_inicio.record()

    # Entrenar una época
    perdida_epoca = entrenar_epoca(
        modelo_generativo,
        cargador_entrenamiento,
        optimizador,
        criterio_perdida,
        indice_padding
    )

    historial_perdidas.append(perdida_epoca)

    # Actualizar learning rate
    scheduler.step()
    lr_actual = scheduler.get_last_lr()[0]

    if tiempo_fin:
        tiempo_fin.record()
        torch.cuda.synchronize()
        tiempo_epoca = tiempo_inicio.elapsed_time(tiempo_fin) / 1000.0
    else:
        tiempo_epoca = 0

    # Mostrar progreso
    print(f"📈 Época {epoca}/{NUM_EPOCAS} | Pérdida: {perdida_epoca:.4f} | LR: {lr_actual:.6f} | Tiempo: {tiempo_epoca:.1f}s")

    # Guardar mejor modelo
    if perdida_epoca < mejor_perdida:
        mejor_perdida = perdida_epoca
        print(f"   ✨ Nueva mejor pérdida! Guardando modelo...")
        torch.save({
            'epoch': epoca,
            'model_state_dict': modelo_generativo.state_dict(),
            'optimizer_state_dict': optimizador.state_dict(),
            'loss': perdida_epoca,
            'vocab_size': tamaño_vocabulario,
            'config': {
                'dim_embedding': DIM_EMBEDDING,
                'num_heads': CABEZAS_ATENCION,
                'num_layers': CAPAS_TRANSFORMER,
                'dropout': DROPOUT
            }
        }, RUTA_GUARDAR_MODELO)

    # Generar muestras cada 10 épocas
    if epoca % 10 == 0:
        print("\n🎯 Generando muestras de ejemplo:")
        for categoria in ["tops", "jeans", "shoes"]:
            texto_muestra = generar_texto_reseña(modelo_generativo, categoria, diccionario_vocabulario, temperatura=0.7)
            print(f"   • {categoria}: {texto_muestra[:80]}...")

print("\n✅ ENTRENAMIENTO COMPLETADO")
print(f"🏆 Mejor pérdida alcanzada: {mejor_perdida:.4f}")

📁 Cargando conjunto de datos...
✓ Total de registros encontrados: 23486
📖 Tamaño del vocabulario: 6709
📚 Tamaño del vocabulario construido: 6709
🔢 Total de lotes por época: 354

🏗️ Construyendo arquitectura del modelo...
📊 Total de parámetros: 19,489,845
📊 Parámetros entrenables: 19,489,845

🚀 INICIANDO PROCESO DE ENTRENAMIENTO
📈 Época 1/45 | Pérdida: 5.2344 | LR: 0.000200 | Tiempo: 95.1s
   ✨ Nueva mejor pérdida! Guardando modelo...
📈 Época 2/45 | Pérdida: 4.4949 | LR: 0.000199 | Tiempo: 98.6s
   ✨ Nueva mejor pérdida! Guardando modelo...
📈 Época 3/45 | Pérdida: 4.2863 | LR: 0.000198 | Tiempo: 101.0s
   ✨ Nueva mejor pérdida! Guardando modelo...
📈 Época 4/45 | Pérdida: 4.1594 | LR: 0.000197 | Tiempo: 101.5s
   ✨ Nueva mejor pérdida! Guardando modelo...
📈 Época 5/45 | Pérdida: 4.0694 | LR: 0.000195 | Tiempo: 101.8s
   ✨ Nueva mejor pérdida! Guardando modelo...
📈 Época 6/45 | Pérdida: 3.9973 | LR: 0.000192 | Tiempo: 101.7s
   ✨ Nueva mejor pérdida! Guardando modelo...
📈 Época 7/45 | Pér


Cantidad de datos:
El modelo fue entrenado con 23,486 registros y un vocabulario de 6,709 tokens. Esto es un tamaño considerable, lo que ayuda a que el modelo generalice mejor.

Parámetros:
El modelo tiene 19,489,845 parámetros entrenables, lo que indica una arquitectura robusta y capaz de aprender patrones complejos.

Pérdida (Loss):
La pérdida inicial en la primera época fue de 5.2344.
La pérdida fue disminuyendo de manera constante en cada época, lo que es una señal de que el modelo está aprendiendo correctamente.
La mejor pérdida alcanzada fue de 3.1146 en la última época (45/45).
Cada vez que la pérdida mejoró, el modelo fue guardado automáticamente.

# Fase de evaluación y generación:
En esta sección se evalúa el modelo entrenado y se generan ejemplos de reseñas. Primero, se generan textos para diferentes categorías y temperaturas, mostrando cómo varía la creatividad del modelo según el parámetro de temperatura. Luego, se realiza una evaluación automática del modelo usando métricas como BLEU y ROUGE sobre varias muestras, y se presentan estadísticas detalladas (percentiles, mínimo y máximo) de los resultados obtenidos. Finalmente, se muestra un resumen con información relevante sobre el modelo, el vocabulario y el dispositivo utilizado.

In [ ]:
# === FASE DE EVALUACIÓN Y GENERACIÓN ===

print("\n🎨 GENERACIÓN DE EJEMPLOS DE RESEÑAS")
print("=" * 60)

# Lista de categorías para probar
categorias_prueba = ["blouses", "dresses", "pants", "sweaters", "jackets"]

print("\n📝 Generando reseñas con diferentes temperaturas:\n")

for temperatura in [0.5, 0.8, 1.0]:
    print(f"\n🌡️ Temperatura = {temperatura}")
    print("-" * 40)

    for categoria in categorias_prueba[:3]:  # Solo las primeras 3 para demostración
        reseña_generada = generar_texto_reseña(
            modelo_generativo,
            categoria,
            diccionario_vocabulario,
            longitud_max=40,
            temperatura=temperatura,
            top_k=40
        )
        print(f"📦 {categoria.upper()}:")
        print(f"   {reseña_generada}\n")

# === EVALUACIÓN COMPLETA DEL MODELO ===
print("\n\n🔬 EVALUACIÓN DETALLADA DEL MODELO")
print("=" * 60)

# Evaluar con métricas automáticas
resultados_metricas = evaluar_calidad_generacion(
    modelo_generativo,
    conjunto_entrenamiento,
    diccionario_vocabulario,
    num_muestras=20,
    temperatura=0.8
)

# Visualización adicional
print("\n\n📊 ANÁLISIS ESTADÍSTICO ADICIONAL")
print("=" * 60)

# Calcular percentiles
for metrica, valores in resultados_metricas.items():
    p25 = np.percentile(valores, 25)
    p50 = np.percentile(valores, 50)
    p75 = np.percentile(valores, 75)
    print(f"\n{metrica.upper()}:")
    print(f"  - P25: {p25:.4f}")
    print(f"  - P50 (mediana): {p50:.4f}")
    print(f"  - P75: {p75:.4f}")
    print(f"  - Min: {min(valores):.4f}")
    print(f"  - Max: {max(valores):.4f}")

# Mensaje final
print("\n\n✨ PROCESO COMPLETADO EXITOSAMENTE ✨")
print(f"📁 Modelo guardado en: {RUTA_GUARDAR_MODELO}")
print(f"📊 Vocabulario final: {tamaño_vocabulario} tokens")
print(f"⚡ Dispositivo utilizado: {DISPOSITIVO}")
print("\n🎉 ¡Gracias por usar este sistema de generación de reseñas!")


🎨 GENERACIÓN DE EJEMPLOS DE RESEÑAS

📝 Generando reseñas con diferentes temperaturas:


🌡️ Temperatura = 0.5
----------------------------------------
📦 BLOUSES:
   i just received this top in the mail and i love it . i am 5 ' 2 ", 120 lbs and 34b and i got the xs . i could probably have gone with an xxs , but i

📦 DRESSES:
   this dress is beautiful and fits true to size . i am 5 ' 5 " and about 120 lbs . i ordered a 4 petite and it fits perfectly . i can ' t wait to wear it !

📦 PANTS:
   i love these pants , they are so comfortable , and stylish . i am 5 ' 10 ", and the length is perfect for me . they are the perfect length for me , they are not too long


🌡️ Temperatura = 0.8
----------------------------------------
📦 BLOUSES:
   i love this top . i bought it in both the black and white colors . i thought it would be a great top to wear for work or play . i did size up because i have broad shoulders

📦 DRESSES:
   the color and cut of this dress are gorgeous , but the fit was so ti

**¿Qué miden estas métricas?**

**BLEU:**
Mide la coincidencia de n-gramas (palabras o secuencias de palabras) entre el texto generado y el de referencia.

**0:** Nada coincide, 1: Coincidencia perfecta.
En generación de texto libre, valores entre 0.02 y 0.10 son normales, ya que hay muchas formas válidas de decir lo mismo.

**ROUGE-1, ROUGE-2, ROUGE-L:**
Miden la superposición de palabras (ROUGE-1), pares de palabras (ROUGE-2) y la subsecuencia común más larga (ROUGE-L) entre el texto generado y el real.
Valores entre 0.1 y 0.4 suelen ser razonables para generación de texto abierto.

**Resultados obtenidos:**
BLEU promedio: 0.0421 (±0.0244)

ROUGE-1 promedio: 0.2820 (±0.0970)

ROUGE-2 promedio: 0.0505 (±0.0403)

ROUGE-L promedio: 0.1764 (±0.0612)

BLEU bajo es normal en generación de texto libre, porque el modelo puede generar frases correctas pero diferentes a la referencia.
ROUGE-1 y ROUGE-L muestran que hay una superposición razonable de palabras y frases entre lo generado y lo real.
ROUGE-2 es más bajo, lo que indica que la coincidencia de pares de palabras exactos es limitada, pero esto es esperable en tareas creativas.

# Función para cargar y usar el modelo entrenado:
En esta sección se incluyen funciones para cargar un modelo previamente entrenado desde un archivo y para realizar inferencias (generar textos nuevos).
La función cargar_modelo_entrenado permite restaurar el modelo y sus pesos desde un checkpoint guardado, recuperando también la configuración y el vocabulario.
La función demo_inferencia muestra cómo utilizar el modelo cargado para generar ejemplos de reseñas a partir de diferentes categorías y subcategorías, demostrando el uso práctico del sistema en modo inferencia.


In [ ]:
# === FUNCIÓN PARA CARGAR Y USAR EL MODELO ENTRENADO ===

def cargar_modelo_entrenado(ruta_modelo, dispositivo=DISPOSITIVO):
    """
    Carga un modelo previamente entrenado desde un archivo checkpoint
    """
    print("📂 Cargando modelo desde checkpoint...")

    # Cargar checkpoint
    checkpoint = torch.load(ruta_modelo, map_location=dispositivo)

    # Extraer configuración
    config = checkpoint['config']
    tamaño_vocab = checkpoint['vocab_size']

    # Recrear modelo
    modelo = ModeloGeneradorTexto(
        tamaño_vocabulario=tamaño_vocab,
        dim_modelo=config['dim_embedding'],
        num_cabezas=config['num_heads'],
        num_capas=config['num_layers'],
        dropout=config['dropout']
    ).to(dispositivo)

    # Cargar pesos
    modelo.load_state_dict(checkpoint['model_state_dict'])
    modelo.eval()

    print(f"✅ Modelo cargado exitosamente")
    print(f"   - Época: {checkpoint['epoch']}")
    print(f"   - Pérdida: {checkpoint['loss']:.4f}")
    print(f"   - Vocabulario: {tamaño_vocab} tokens")

    return modelo

# Ejemplo de uso para inferencia
def demo_inferencia():
    """
    Demostración de cómo usar el modelo para inferencia
    """
    # Cargar modelo (descomentar cuando exista el archivo)
    # modelo_cargado = cargar_modelo_entrenado(RUTA_GUARDAR_MODELO)

    print("\n🎯 MODO INFERENCIA - Ejemplos de uso")
    print("=" * 60)

    # Categorías de ejemplo
    categorias_demo = {
        "tops": ["casual", "formal", "summer"],
        "dresses": ["party", "casual", "wedding"],
        "shoes": ["running", "formal", "boots"]
    }

    # Generar ejemplos (usando el modelo actual del entrenamiento)
    for categoria_principal, subcategorias in categorias_demo.items():
        print(f"\n📦 {categoria_principal.upper()}")
        for sub in subcategorias[:2]:  # Solo 2 ejemplos por categoría
            prompt = f"{sub} {categoria_principal}"
            reseña = generar_texto_reseña(
                modelo_generativo,  # Cambiar a modelo_cargado cuando se use el checkpoint
                prompt,
                diccionario_vocabulario,
                longitud_max=35,
                temperatura=0.85,
                top_k=30
            )
            print(f"  • {sub}: {reseña}")

# Ejecutar demostración
demo_inferencia()



🎯 MODO INFERENCIA - Ejemplos de uso

📦 TOPS
  • casual: love this little number ! the material is soft and the length is perfect for me . it is a bit of a baggy fit , but that ' s what makes it so fun
  • formal: i was really bummed about this top except for the price . it really is absolutely beautiful in person , and i couldn ' t bring myself to figure out how to wear it .

📦 DRESSES
  • party: love this dress ! it is so comfy and fits as shown . i am 5 ' 9 " and this dress hits me a little below the knee , but it is actually quite
  • casual: i bought this dress in the blue stripe . it ' s very flattering and comfy . it is a bit long ( i ' m 5 ' 1 ) but i love it nonetheless

📦 SHOES
  • running: the first time i wore this , i got the blue color back and its a lovely shade of blue . it ' s not too see - through , a cami underneath is a
  • formal: great material . i really wanted to love this but unfortunately i will be returning it .


# Conclusión

El modelo Transformer entrenado para la generación de reseñas ha mostrado un desempeño sólido y coherente. Durante el entrenamiento, la pérdida disminuyó de forma constante, indicando un buen aprendizaje. Los textos generados son naturales y relevantes para cada categoría, y la variación de la temperatura permite ajustar la creatividad de las respuestas.

Las métricas automáticas (BLEU y ROUGE) se encuentran en rangos esperados para tareas de generación de texto libre, confirmando que el modelo no memoriza, sino que generaliza y produce reseñas plausibles y variadas. En resumen, el sistema es robusto, versátil y está listo para ser utilizado en aplicaciones reales o como base para futuras mejoras.